# Fetch-API.ipynb
## Fetches YouTube Most Viewed Videos in a Region and Saves to a CSV.
### ! Don't forget to put your YouTube Data API Key in API_KEY !

### Libraries

In [1]:
import urllib.request, json # To fetch YouTube Data Json.
import pandas as pd # To make the CSV file.

### Global Variables

In [8]:
NUMBER_OF_VIDEOS_PER_PAGE = "50" # Number of videos per page.
NUMBER_OF_PAGES = 1 # Number of pages with NUMBER_OF_VIDEOS_PER_PAGE. (Over than 8 usually bugs the YouTube API)
REGION = "CA" # Region to get Most View Videos.
API_KEY = "" # Enter YOUR API Key Here.
EXPORT_PATH = r'C:\Youtube\CA_viewCount.csv' # Path to save CSV file.
NEXT_PAGE = "" #Provide Next Page Token to Restart From Page.

###  ! Do not run more than once if you want to keep old JSON requests !

In [9]:
data = [] # Contains JSON requests.

### Fetches Videos using YouTube Data API (Run this multiple times to get many pages)

In [29]:
for i in range(NUMBER_OF_PAGES):
    if not data:
        if NEXT_PAGE == "":
            with urllib.request.urlopen("https://www.googleapis.com/youtube/v3/search?part=snippet&order=viewCount&type=video&regionCode=" + REGION + "&maxResults=" + NUMBER_OF_VIDEOS_PER_PAGE + "&key=" + API_KEY) as url:
                data.append(json.loads(url.read().decode()))
        else:
            with urllib.request.urlopen("https://www.googleapis.com/youtube/v3/search?part=snippet&order=viewCount&type=video&pageToken=" + NEXT_PAGE + "regionCode=" + REGION + "&maxResults=" + NUMBER_OF_VIDEOS_PER_PAGE + "&key=" + API_KEY) as url:
                data.append(json.loads(url.read().decode()))
    else:
        with urllib.request.urlopen("https://www.googleapis.com/youtube/v3/search?part=snippet&order=viewCount&type=video&pageToken=" + data[-1]["nextPageToken"] + "&chart=mostPopular&regionCode=" + REGION + "&maxResults=" + NUMBER_OF_VIDEOS_PER_PAGE + "&key=" + API_KEY) as url:
            data.append(json.loads(url.read().decode()))
    print(data[-1]["nextPageToken"])

KeyError: 'nextPageToken'

### Cleans up the JSON Data and Make a Pandas DataFrame

In [43]:
video_dictionary = {'id':[], 'title': [], 'description': [], 'thumbnails': []}

for pages in data[:-11]:
    for video in pages["items"]:
        video_dictionary["id"].append(video["id"]["videoId"])
        video_dictionary["title"].append(video["snippet"]["title"])
        video_dictionary["thumbnails"].append(video["snippet"]["thumbnails"])
        video_dictionary["description"].append(video["snippet"]["description"])
        
video_database = pd.DataFrame(video_dictionary)

### Write Pandas DataFrame to CSV File

In [44]:
video_database.to_csv(EXPORT_PATH, index = None, header=True)